# MPR121 Data Recording Notebook

Author: Christopher Parker (parkecp@mail.uc.edu)

I've tried to make this as user-friendly as possible, but feel free to reach out with any questions.

### Imports

In [1]:
# Basic libraries
import os
os.environ['BLINKA_MPR121'] = '1'
os.environ['BLINKA_FT232H'] = '1'
import time
import datetime
from collections import deque

# For writing data to file
import h5py

# Libraries for FTDI and I2C
from pyftdi.i2c import I2cController
from pyftdi.usbtools import UsbTools

# Multiprocessing
from concurrent.futures import ThreadPoolExecutor

### Device Configuration and Constant Declarations

Before we start recording, we need to configure the MPR121s.

Normally, this would be done automatically by the Adafruit Blinka package, but we cannot use that (because it only allows 1 FT232H at a time).
So instead, we have to use the pyftdi package (which is what Adafruit Blinka uses, also) and manually determine which registers need to be written/read.

In [2]:
# Register Addresses
SOFT_RESET = 0x80
CONFIG = 0x5E
DATA = 0x04

# How many sensor samples we want to store before writing
HISTORY_SIZE = 1000

# 12 channels per MPR121, just defining a constant
# for clarity
NUM_CHANNELS = 12

In [3]:
# Find all devices with the given vendor and product IDs
# These correspond to the FT232H
devices = UsbTools.find_all([(0x0403, 0x6014)])

NUM_SENSORS = len(devices)

In [4]:
# We will make lists of the I2C controller objects and port objects
# for each FT232H device
i2c_controllers = []
i2c_ports = []
for dev in devices:
    url = f"ftdi://ftdi:232h:{dev[0].bus}:{dev[0].address}/1"
    controller = I2cController()
    controller.configure(url)
    i2c_controllers.append(controller)
    port = controller.get_port(0x5A)
    # I really need to figure out a better way to determine the correct address,
    # because this looks silly
    try: # Try to read a byte to see if that's the right address
        port.read_from(0x04, 1)
    except: # If not, try again
        port = controller.get_port(0x5B)
    try:
        port.read_from(0x04, 1)
    except:
        port = controller.get_port(0x5C)
    try:
        port.read_from(0x04, 1)
    except:
        port = controller.get_port(0x5D)
    try:
        port.read_from(0x04,1)
    except:
        print('None of the MPR121 addresses is readable, is the FT232H in I2C mode?')
    i2c_ports.append(port)

In [5]:
# Loop over the FT232Hs, reset and configure each
for idx, port in enumerate(i2c_ports):
    # Write to Soft Reset Register (0x63 sends reset command)
    port.write_to(SOFT_RESET, b'\x63')
    # 0x5E is the configuration register, setting to 0x8F starts the MPR121
    # with the config used in the Adafruit library (if needed, I can figure
    # out alternative configurations)
    port.write_to(CONFIG, b'\x8F')
    
    # If we don't sleep here, it doesn't have time to start reading properly
    time.sleep(0.1)
    
    # Test that we have started the MPR121 (if it's not properly started,
    # reading from DATA will give an empty bytearray, so we check if that's
    # what we got back)
    cap = port.read_from(DATA, 24)
    if cap != bytearray(24):
        print(f'Sensor {idx} Started')

Sensor 0 Started
Sensor 1 Started


### Widget Setup

#### Recording Function

In [6]:
def record():
    # Time data (for each channel, because they are updated independently)
    # We use deques here because they are more efficient for appending and popping
    time_data = deque(maxlen=12)
    # Capacitance data (again for each channel)
    cap_data = deque(maxlen=12)
    while not i2c.try_lock():
        pass
    try:
        i2c.writeto_then_readfrom(mpr121_address, bytes([start_reg]), raw_buffer)
    finally:
        i2c.unlock()

    # Process the raw data for each electrode
    for chan in range(12):
        # Combine the two bytes (little-endian) for each electrode
        value = raw_buffer[2 * chan] | (raw_buffer[2 * chan + 1] << 8)

        # Save the value to the cap_data list and the current time to the time_data deque
        cap_data.append(value)            
        time_data.append(time.time())
    return time_data, cap_data

## Start Recording Here

In [7]:
def record(i2c_port):
    """
    Reads 24 bytes (2 bytes per channel for 12 channels) from an MPR121 sensor 
    via the given pyftdi I2C port, and returns timestamp and capacitance data.
    """
    local_time_data = deque(maxlen=NUM_CHANNELS)
    local_cap_data = deque(maxlen=NUM_CHANNELS)

    # Read 24 bytes (2 bytes for each of the 12 channels).
    raw_buffer = i2c_port.read_from(DATA, 24)
    for chan in range(NUM_CHANNELS):
        # Combine two bytes (little-endian)
        value = raw_buffer[2 * chan] | (raw_buffer[2 * chan + 1] << 8)
        local_cap_data.append(value)
        local_time_data.append(time.time())
    return local_time_data, local_cap_data


In [8]:
%%time
sensor_time_data = {
    sensor: [deque(maxlen=HISTORY_SIZE) for _ in range(NUM_CHANNELS)]
    for sensor in range(NUM_SENSORS)
}
sensor_cap_data = {
    sensor: [deque(maxlen=HISTORY_SIZE) for _ in range(NUM_CHANNELS)]
    for sensor in range(NUM_SENSORS)
}
# Alternatives for stopping the loop (iterations or time),
# also we are writing to file every 1000 reads
stop = False # Flag to stop looping
loop_ctr = 0
cutoff_time = time.time() + 60 * 60 * 2  # Run for 2 hours

# Open an HDF5 file to store the data.
filename = f"raw_data_{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.h5"
with h5py.File(filename, "w") as h5f, ThreadPoolExecutor(max_workers=NUM_SENSORS) as executor:
    # Create a group for each sensor and initialize datasets.
    sensor_groups = {}
    for sensor in range(NUM_SENSORS):
        grp = h5f.create_group(f"sensor_{sensor}")
        sensor_groups[sensor] = grp

    while not stop:
        start_time = time.time()
        
        # Launch parallel sensor reads on all I2C ports.
        futures = [executor.submit(record, port) for port in i2c_ports]
        results = [future.result() for future in futures]  # Each result is (local_time_data, local_cap_data)
        # Append the data from each sensor to its corresponding deques.
        for sensor_idx, (local_time, local_cap) in enumerate(results):
            for chan in range(NUM_CHANNELS):
                sensor_time_data[sensor_idx][chan].append(local_time[chan])
                sensor_cap_data[sensor_idx][chan].append(local_cap[chan])
        loop_time = time.time()
        #print(f"Loop time: {loop_time - start_time}")
        if loop_ctr == HISTORY_SIZE:
            for sensor, group in sensor_groups.items():
                group.create_dataset("time_data", data=sensor_time_data[sensor], chunks=(NUM_CHANNELS, HISTORY_SIZE), maxshape=(NUM_CHANNELS, None))
                group.create_dataset("cap_data", data=sensor_cap_data[sensor], chunks=(NUM_CHANNELS, HISTORY_SIZE), maxshape=(NUM_CHANNELS, None))
            # for sensor in range(NUM_SENSORS):
            #     sensor_groups[sensor].create_dataset("time_data", data=sensor_time_data[sensor], chunks=(NUM_CHANNELS, HISTORY_SIZE), maxshape=(NUM_CHANNELS, None))
            #     sensor_groups[sensor].create_dataset("cap_data", data=sensor_cap_data[sensor], chunks=(NUM_CHANNELS, HISTORY_SIZE), maxshape=(NUM_CHANNELS, None))
        elif loop_ctr != 0 and loop_ctr%HISTORY_SIZE == 0:
            tmp_ctr = loop_ctr - 1000
            for sensor, group in sensor_groups.items():
                group["time_data"].resize((NUM_CHANNELS, tmp_ctr + HISTORY_SIZE))
                group["cap_data"].resize((NUM_CHANNELS, tmp_ctr + HISTORY_SIZE))
                group["time_data"][:, tmp_ctr:tmp_ctr + HISTORY_SIZE] = sensor_time_data[sensor]
                group["cap_data"][:, tmp_ctr:tmp_ctr + HISTORY_SIZE] = sensor_cap_data[sensor]
        # Stop after a set number of reads
        if loop_ctr == 2000:
            stop = True
        if time.time() >= cutoff_time:
            stop = True
        loop_ctr += 1

CPU times: user 4.04 s, sys: 4.24 s, total: 8.27 s
Wall time: 40.2 s
